# Use your own data

In this tutorial, we discuss two ways of using your own data:

1. You have one or more lexicons you want to evaluate and generate streams with
2. You already have streams and just want to evaluate them

If you want to expand ARC, we are happy to invite you to contribute to the [ARC Project](https://github.com/milosen/arc) 


## 1. Loading/creating your custom lexicon

Let's say you have a lexicon consisting of the (pseudo-)words 'piɾuta', 'baɡoli', 'tokuda, and 'ɡuhaɪbo'.

We assume you have prepared your lexicon as a list of lists (see below), and that all syllables are of the same type. The function `to_lexicon()` accepts the syllable types we call 'cv' and 'cV'. 'cv' is a syllable consisting of a single-character consonant and a short vowel, e.g. 'pi'. Because it is common in the literature, 'cv' also allows diphthongs, e.g. 'haɪ'). The 'cV' type is a single-character consonant, together with a long vowel, e.g. 'tuː'.

In [9]:
from arc import to_lexicon
import numpy as np

raw_lexicon = [
  ['pi', 'ɾu', 'ta'],
  ['ba', 'ɡo', 'li'],
  ['to', 'ku', 'da'],
  ['ɡu', 'haɪ', 'bo']
]

lexicon = to_lexicon(raw_lexicon, syllable_type="cv")

print("Lexicon:", lexicon)
print("")

for key, value in lexicon.info.items():
    print(f"{key}:", lexicon.info[key])


Lexicon: piɾuta|baɡoli|tokuda|ɡuhaɪbo

syllables_info: {'syllable_feature_labels': [['son', 'back', 'hi', 'lab', 'cor', 'cont', 'lat', 'nas', 'voi'], ['back', 'hi', 'lo', 'lab', 'tense', 'long']], 'syllable_type': 'cv'}
cumulative_feature_repetitiveness: 7
max_pairwise_feature_repetitiveness: 2


### 1.1. Custom Lexicon: Moving upstream

Now we "move upstream" in the generation process. We turn the lexicon into a stream using the standard `arc` functions introduced earlier.

In [10]:
from arc import make_streams
streams = make_streams([lexicon])

print("Streams (summary):", streams)
print("")

for key, value in streams.info.items():
    print(f"{key}:", value)

Streams (summary): piɾutabaɡolitokudaɡuhaɪbo_random|piɾutabaɡolitokudaɡuhaɪbo_word_structured|piɾutabaɡolitokudaɡuhaɪbo_position_controlled

tp_modes: ('random', 'word_structured', 'position_controlled')
max_rhythmicity: None
max_tries_randomize: 10
stream_length: 32
require_all_tp_modes: True


In [11]:
for stream in streams:
    tp_mode = stream.info['stream_tp_mode']
    pris = stream.info['rhythmicity_indexes']
    
    print(f"Stream ({tp_mode}): ", stream)
    print("PRIs:")
    max = "phon_1_son"
    cum = 0
    for feat, pri in stream.info["rhythmicity_indexes"].items():
        print(" ", feat, pri)
        if pri > stream.info["rhythmicity_indexes"][max]:
            max = feat
        cum += pri

    print("Max PRI across features:", max, stream.info["rhythmicity_indexes"][max])
    print("Cummulative PRI across features:", cum)
    print(" ")

Stream (random):  ku|pi|li|bo|ɡu|to|haɪ|ba|da|ɾu|ɡo|ta|ɡo|da|ku|ta|ɾu|ba|li|to|bo|pi|haɪ|ɡu|ba|bo|haɪ|da|ta|pi|to|li|ɾu|ɡu|ku|ɡo|ba|ɡu|ɾu|pi|bo|li|ɡo|ku|to|da|haɪ|ta|ɡu|li|da|to|ɾu|bo|ku|ba|haɪ|ɡo|pi|ta|li|ta|ku|ɾu|to|ɡo|haɪ|pi|ɡu|bo|da|ba|ta|ba|pi|ku|haɪ|bo|ɾu|li|ɡu|da|ɡo|to|ta|bo|ɡo|ɡu|pi|ɾu|haɪ|to|ba|ku|da|li|ba|ɡo|bo|to|ku|li|pi|da|ɡu|ta|haɪ|ɾu|da|bo|ba|ɾu|ta|to|pi|ɡo|li|haɪ|ku|ɡu|ɡo|ɾu|ku|bo|ta|da|pi|ba|to|ɡu|haɪ|li|ku|ɡu|pi|li|ɾu|haɪ|to|ɡo|ta|ba|bo|da|li|ɡu|ɡo|bo|to|ku|da|pi|haɪ|ba|ta|ɾu|to|li|pi|bo|ɡu|ba|ɡo|ɾu|ku|haɪ|da|ta|li|ba|da|ɾu|ɡu|haɪ|ta|ku|bo|ɡo|to|pi|ɡu|li|ɡo|pi|ɾu|da|bo|ba|to|ta|haɪ|ku|li|bo|haɪ|ɾu|ta|ɡu|ku|ɡo|ba|pi|da|to|da|ku|pi|to|haɪ|ɡo|ɡu|bo|ɾu|ba|li|ta|ɡo|da|ɡu|ta|to|bo|ku|ɾu|pi|ba|haɪ|li|haɪ|pi|ɡo|li|ku|ba|ɡu|to|ɾu|bo|ta|da|haɪ|ɡu|da|ba|ɾu|ɡo|ku|to|ta|pi|li|bo|li|da|ɡo|haɪ|bo|pi|ta|ba|ku|ɾu|ɡu|to|ɡu|ɾu|li|to|ba|haɪ|da|pi|ku|ta|bo|ɡo|ɾu|ɡo|ɡu|bo|li|ba|da|ta|haɪ|to|pi|ku|ɡo|ku|ta|bo|haɪ|ɾu|to|da|ɡu|pi|ba|li|ɡo|ta|ɾu|li|to|bo|ɡu|ku|da|ba|pi|haɪ|ku|ɡu|ba|ɾu|da|bo|to

### 1.2. Custom Lexicon: Moving backwards

"moving backwards" in the generation process, i.e. generating words, syllables, and phonemes is less common, but we got you covered. Let's say you want to compare the syllables in your custom lexicon with the arc corpus.

In [12]:
syllables = lexicon.flatten()
print(syllables)

pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|... (12 elements total)


In [13]:
from arc.io import read_syllables_corpus

corpus_syllables = read_syllables_corpus()

syllables_with_corpus_stats = syllables.intersection(corpus_syllables)

print(syllables_with_corpus_stats)
syllables_with_corpus_stats["pi"].info

#note: mention that frew and prob are new

pi|ta|ba|ɡo|li|to|ku|da|ɡu|haɪ|... (11 elements total)


{'binary_features': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 'phonotactic_features': [['plo', 'lab'], ['i']],
 'freq': 70,
 'prob': 6.92628e-05}

In [14]:
phonemes = syllables.flatten()
print(phonemes)

p|i|ɾ|u|t|a|b|ɡ|o|l|... (13 elements total)


In [15]:
from arc.io import read_phoneme_corpus
corpus_phonemes = read_phoneme_corpus()

phonemes_with_stats = phonemes.intersection(corpus_phonemes)
print(phonemes_with_stats)
print(phonemes_with_stats["p"].info)

p|t|a|b|ɡ|l|k|d|h
{'features': ['-', '-', '+', '-', '-', '-', '-', '0', '-', '-', '-', '+', '-', '0', '+', '-', '-', '-', '-', '0', '-'], 'word_position_prob': {0: 0.17205350479361617, 1: 0.21734887192146507, 2: 0.28055571502382454, 3: 0.19685400998909236, 4: 0.04449164705206958, 5: 0.03743039210057983, 6: 0.018542970319765772, 7: 0.012859521212469143, 8: 0.007463114989379413, 9: 0.003961191802055227, 10: 0.003387106033641426, 11: 0.0014926229978758827, 12: 0.0020667087662896836, 13: 0.0006314943452551811, 14: 0.00040186003788966073, 15: 0.0002870428842069005, 16: 5.74085768413801e-05, 17: 0.0, 18: 5.74085768413801e-05, 19: 5.74085768413801e-05}}


## Evaluating your stream

Again, we assume you have prepared your data into a list of syllables like below.

In [16]:
from arc import to_stream

stream = ['pi', 'ɾu', 'ta', 'ba', 'ɡo', 'li', 'to', 'ku', 'da', 'ɡu', 'ki', 'bo']*streams.info['stream_length']

stream = to_stream(stream)

print("Stream: ", stream, end="\n\n")
print("rhythmicity indexes (PRIs)", stream.info['rhythmicity_indexes'])

Stream:  pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|to|ku|da|ɡu|ki|bo|pi|ɾu|ta|ba|ɡo|li|t

As you can see, even with a custom lexicon, the randomization of a stream has an effect on the PRIs.

This concludes our third and last tutorial. We hope you feel ready to use ARC, and help us extend it.